
# N3 Configuration with varying outer air region thickness and Wavelength

Here we create (hopefully) an interactive plot to see what happens to the modes as we vary the outer air region thickness.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf, plotlogf_real
from ngsolve.webgui import Draw
from scipy.optimize import newton

from ipywidgets import interactive, FloatSlider, Layout

importing NGSolve-6.2.2203-56-g47f483033


In [2]:
n_air = 1.00027717
n_glass = 1.4388164768221814

ns = [lambda x:n_air, lambda x:n_glass, lambda x:n_air, lambda x : n_glass, lambda x: n_air]
mats = ['air', 'glass','air', 'glass', 'air']
maxhs = [100 for r in mats]
outer = 'h2'

nu = 1

d = 4.0775e-05
ts = [d , 1e-5, d, 1e-5, 2e-5]
H = BraggExact(ts=ts,ns=ns, mats=mats, maxhs=maxhs, no_mesh=True)

# Both Wavelength and T

In [14]:
nu = 1

def det_plot(T, wl):
    d = 4.0775e-05  # thickness of innermost region (core radius)
    ts = [d , 1e-5, T*d, 1e-5, 2e-5]
    H.ts = ts
    H.wavelength = wl * 1e-6
    cutoff = H.ks[0] * H.scale
    
    Xr = np.linspace(.9998* cutoff , 1.00005*cutoff , num=75)
    Xi = np.linspace( -.005, .0015, num=75)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig, ax1 = plt.subplots(1, 1, figsize=(14,10))
    
    fs1 = H.determinant(zs, nu=nu, outer='h2')
    ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=75)
    ax1.grid(True)
    m, M = ax1.get_ylim()
    ax1.plot([cutoff, cutoff], [m,M],lw=1.2, ls='--', c="orange")
    ax1.set_facecolor('grey')
    ax1.set_title('Behavior near Fundamental Mode as buffer air thickness goes to zero')
#     ax1.set_xticklabels([])
    ax1.set_yticks([0])
    plt.show()
    
interactive_plot = interactive(det_plot,
                               T=FloatSlider(min=0, max=.003, step=.00001, value=.00045, 
                                             readout_format='.5f', layout=Layout(width='100%')),
                               wl=FloatSlider(min=1.4, max=2, step=.02, value=1.8,
                                              readout_format='.2f', layout=Layout(width='100%')))

output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

interactive(children=(FloatSlider(value=0.00045, description='T', layout=Layout(width='100%'), max=0.003, read…

In [25]:
A = BraggExact(ts=H.ts)


In [26]:
Draw(A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

# The Bumping is much more dramatic at some wavelengths.

At 1.89, the fundamental runs from the incoming modes right away as T increases from 1.

Would be worth looking at mode profiles for this.

In [41]:
k_low = H.ks[0] * H.scale
guess = .999999*k_low

In [42]:
from scipy.optimize import newton
guess = np.array(guess)

beta1 = newton(H.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(H.determinant(beta1, nu, outer)))


Scaled beta:  (49.86650019043079-2.436814209760855e-27j) . Residual of determinant:  226.9337091789876


In [43]:
Fs = H.all_fields(beta1, nu, outer)

In [44]:
Draw(Fs['Ez'], A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

# Same mode profile pattern holds

The bumped mode goes from fundamental to large area, middle mode becomes new fundamental, left mode is sort of blend.